gcn = graph convolution network
    = gnn + normalizaiton

custom layer

In [7]:
import torch
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root=".", name="Cora")
data = dataset[0]

# 정확도 함수
def accuracy(y_pred, y_true):
    """Calculate accuracy."""
    return torch.sum(y_pred == y_true) / len(y_true)

In [8]:
# gcn layer
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import to_dense_adj
# Basic GNN
class GCNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        
        # bias를 포함하지 않는 basic linear transformation
        self.linear = nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, edge_index):
        #num_nodes = x.size(0)
        adjacency = to_dense_adj(edge_index)[0]
        adjacency += torch.eye(len(adjacency))
        degree_matrix = torch.diag(adjacency.sum(dim=1))
        degree_inv_sqrt = torch.diag(torch.pow(torch.diag(degree_matrix), -0.5)) # D^(-0.5)
        adj_norm = degree_inv_sqrt @ adjacency @ degree_inv_sqrt
        # (1) Linear trasnformation
        x = self.linear(x) # hidden layer (X*W.T)
        
        # (2) Multiplication with the adjacency matrix A
        x = torch.sparse.mm(adj_norm, x) # H = A.T (X*W.T), A = D^(-0.5)AD^(-0.5)
        return x

In [9]:
class CustomGCN(torch.nn.Module):
    """Graph Convolutional Network"""
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNLayer(dim_in, dim_h)
        self.gcn2 = GCNLayer(dim_h, dim_out)
 
    def forward(self, x, edge_index):
        h = self.gcn1(x, edge_index)
        h = torch.relu(h)
        h = self.gcn2(h, edge_index)
        return F.log_softmax(h, dim=1)
 
    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.01,
                                      weight_decay=5e-4)
 
        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1),
                          data.y[data.train_mask])
            loss.backward()
            optimizer.step()
 
            if(epoch % 20 == 0):
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1),
                                  data.y[data.val_mask])
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc:'
                      f' {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | '
                      f'Val Acc: {val_acc*100:.2f}%')
 
    @torch.no_grad()
    def test(self, data):
        self.eval()
        out = self(data.x, data.edge_index)
        acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
        return acc


In [10]:
# 인스턴스 저장 및 100 에포크 훈련
gcn = CustomGCN(dataset.num_features, 16, dataset.num_classes)
print(gcn)
 
# Train
gcn.fit(data, epochs=100)
 
# Test
acc = gcn.test(data)
print(f'\nGCN test accuracy: {acc*100:.2f}%\n')

CustomGCN(
  (gcn1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=False)
  )
  (gcn2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=False)
  )
)
Epoch   0 | Train Loss: 1.944 | Train Acc: 18.57% | Val Loss: 1.94 | Val Acc: 15.60%
Epoch  20 | Train Loss: 0.227 | Train Acc: 98.57% | Val Loss: 0.86 | Val Acc: 78.80%
Epoch  40 | Train Loss: 0.022 | Train Acc: 100.00% | Val Loss: 0.73 | Val Acc: 78.40%
Epoch  60 | Train Loss: 0.020 | Train Acc: 100.00% | Val Loss: 0.71 | Val Acc: 78.60%
Epoch  80 | Train Loss: 0.022 | Train Acc: 100.00% | Val Loss: 0.70 | Val Acc: 78.00%
Epoch 100 | Train Loss: 0.019 | Train Acc: 100.00% | Val Loss: 0.71 | Val Acc: 78.20%

GCN test accuracy: 80.40%

